In [19]:
from __future__ import unicode_literals, print_function

import math

from ofblockmeshdicthelper import BlockMeshDict, Vertex, SimpleGrading


In [20]:
# geometries
wt_length = 20 # wt  = wind tunnel x-dir
wt_depth = 3 # z-dir
wt_height = 3 # y-dir
box_size = 1

In [21]:


# prepare ofblockmeshdicthelper.BlockMeshDict instance to
# gather vertices, blocks, faces and boundaries.
bmd = BlockMeshDict()

# set metrics
bmd.set_metric('m')

# base vertices of wind tunnel 
basevs = [
    Vertex(0, 0, 0, 'v0'),
    Vertex(wt_length, 0, 0, 'v1'),
    Vertex(wt_length,  0, wt_depth, 'v2'),
    Vertex(0, 0, wt_depth, 'v3')]

In [22]:
for v in basevs:
    print(f"{v.name}: {v.x, v.y, v.z}")

v0: (0, 0, 0)
v1: (20, 0, 0)
v2: (20, 0, 3)
v3: (0, 0, 3)


In [23]:
# raise up in the y direction 
for v in basevs:
    bmd.add_vertex(v.x, wt_height, v.z, v.name+'+y')
    bmd.add_vertex(v.x, 0, v.z, v.name+'-y')

In [24]:
bmd.vertices

{'v0+y': <ofblockmeshdicthelper.Vertex at 0x107b35720>,
 'v0-y': <ofblockmeshdicthelper.Vertex at 0x107b34f70>,
 'v1+y': <ofblockmeshdicthelper.Vertex at 0x107b34ac0>,
 'v1-y': <ofblockmeshdicthelper.Vertex at 0x107b36d10>,
 'v2+y': <ofblockmeshdicthelper.Vertex at 0x107b36d70>,
 'v2-y': <ofblockmeshdicthelper.Vertex at 0x107b34fd0>,
 'v3+y': <ofblockmeshdicthelper.Vertex at 0x107b34850>,
 'v3-y': <ofblockmeshdicthelper.Vertex at 0x107b365c0>}

In [25]:
def vnamegen(x0z0, x1z0, x1z1, x0z1):
    return (x0z0+'-y', x1z0+'-y', x1z0+'+y', x0z0+'+y',
            x0z1+'-y', x1z1+'-y', x1z1+'+y', x0z1+'+y')

In [26]:

b0 = bmd.add_hexblock(vnamegen('v0', 'v1', 'v2', 'v3'),
                      (80, 50, 20),
                      'b0',
                      grading=SimpleGrading(1,
                                            1,
                                            1))

In [27]:
# checking if this maesses things up... no it doesnt 

# boundaries = ["symmetry", "symmetry", "wall", "patch", "patch", "patch"]
# names = ["side01", "side02", "bottom", "top", "inlet", "outlet"]
# for i, boundary, name in zip(range(6), boundaries, names):
#     bmd.add_boundary(boundary, name, [b0.face(i)])

In [28]:
bmd.add_boundary('wall', 'bottom', [b0.face('s')]) ## should probably be patches?
bmd.add_boundary('patch', 'top', [b0.face('n')])

bmd.add_boundary('patch', 'outlet', [b0.face('e')])
bmd.add_boundary('patch', 'inlet', [b0.face('w')])

bmd.add_boundary('symmetry', 'side01', [b0.face('b')])
bmd.add_boundary('symmetry', 'side02', [b0.face('t')])


In [29]:
print([v.name for v in bmd.boundaries.values()])

['side01', 'side02', 'bottom', 'top', 'inlet', 'outlet']


### internal block

In [30]:
# now want internal block
# base vertices of wind tunnel 
boxvs = [
    Vertex(box_size, 0, box_size, 'v4'),
    Vertex(box_size + box_size, 0, box_size, 'v5'),
    Vertex(box_size + box_size,  0, box_size + box_size, 'v6'),
    Vertex(box_size, 0, box_size + box_size, 'v7')]

In [31]:
# raise up in the y direction 
for v in boxvs:
    bmd.add_vertex(v.x, box_size, v.z, v.name+'+y')
    bmd.add_vertex(v.x, 0, v.z, v.name+'-y')

In [32]:
b1 = bmd.add_hexblock(vnamegen('v4', 'v5', 'v6', 'v7'),
                      (2, 5, 5),
                      'b1',
                      grading=SimpleGrading(1,
                                            1,
                                            1))

In [33]:
b1.face(0).name

'f-b1-w'

In [34]:
[b1.face(i).name for i in range(6)]

['f-b1-w', 'f-b1-n', 'f-b1-s', 'f-b1-n', 'f-b1-b', 'f-b1-t']

In [35]:
bmd.add_boundary('wall', 'box', [b1.face(i) for i in range(6) ]) 

### finish 

In [36]:
bmd.assign_vertexid()

In [37]:
print(bmd.format())

/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2.3.0                                 |
|   \\  /    A nd           | Web:      www.OpenFOAM.org                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 1;

vertices
(
    (                  0                  0                  0 )  // 0 v0-y
    (                 20                  0                  0 )  // 1 v1-y
    (                 20                  3                 